In [1]:
import pandas as pd
import unicodedata
import re
import os

In [4]:
def normalize_string(text):
    """Elimina caracteres especiales y convierte el texto a minúsculas."""
    text = str(text)
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^a-z0-9\s]', '', text.lower()).strip()

class SheetData:
    def __init__(self, data_path: str, sheet_name: str, name_column: str, email_column: str) -> None:
        self.sheet_name = sheet_name
        self._df = pd.read_excel(data_path, sheet_name=sheet_name)
        self.name_column = name_column
        self.email_column = email_column
        self.filter_column = None
        self._columns = {
            name_column: "Nombre",
            email_column: "Correo"
        }
    
    def set_filter_column(self, column: str) -> None:
        self.filter_column = column
    
    def filter(self, category: str, options: list[str] = [], column = None, other = None):
        columns = self._columns.copy()
        if column is None:
            column = self.filter_column
        def filter_function(value: str) -> bool:
            for option in options:
                if normalize_string(option) in normalize_string(value):
                    return True
            return False

        if other is not None:
            columns[other] = "Otro"
        if column is None:
            filtered_data = self._df
        else:
            columns[column] = "Origen"
            filtered_data = self._df[self._df[column].apply(filter_function)]

        #filtered_data = filtered_data.rename(columns=columns)[list(columns.values())]
        filtered_data = filtered_data[list(columns.keys())].rename(columns=columns)

        if other is None:
            filtered_data["Otro"] = ""
        if column is None:
            filtered_data["Origen"] = self.sheet_name
        filtered_data["Categoria"] = category
    
        filtered_data = filtered_data[["Nombre", "Correo", "Otro", "Origen", "Categoria"]]
        return filtered_data

class ComunicationData:
    def __init__(self, csv_file: str) -> None:
        self.csv_file = csv_file
        self._columns = ["Nombre", "Correo", "Otro", "Origen", "Categoria"]

        # Si el archivo CSV existe, cargarlo. Si no, crear un DataFrame vacío con las columnas.
        if os.path.exists(self.csv_file):
            self.df = pd.read_csv(self.csv_file)
        else:
            self.df = pd.DataFrame(columns=self._columns)

    def append(self, new_data: pd.DataFrame) -> None:
        # Verificar que el DataFrame tenga las mismas columnas
        if list(new_data.columns) != self._columns:
            raise ValueError(f"Las columnas deben ser {self._columns}y son {list(new_data.columns)}")

        # Eliminar filas con valores vacíos o nulos en las columnas "Nombre" o "Correo"
        new_data = new_data.dropna(subset=["Nombre", "Correo"])

        # Formatear: eliminar saltos de línea y espacios en blanco al inicio y al final de cada campo
        new_data = new_data.map(lambda x: str(x).strip().replace("\n", " ").replace("\r\n", "") if isinstance(x, str) else x)

        # Filtrar las filas que no tengan correos repetidos
        existing_emails = self.df["Correo"].values
        non_duplicate_data = new_data[~new_data["Correo"].isin(existing_emails)]

        # Agregar las filas nuevas al DataFrame actual
        self.df = pd.concat([self.df, non_duplicate_data], ignore_index=True)

        # Guardar el DataFrame actualizado en el archivo CSV
        self.df.to_csv(self.csv_file, index=False)

    

In [5]:
# Datos
amigos_visibles = "data/amigos_visibles.xlsx"
construccion = "data/construccion.xlsx"
afrus = "data/afrus.xlsx"
donaciones_av = "data/donaciones_av.xlsx"
connect = "data/connect.xlsx"
aliados = "data/aliados.xlsx"
evenbrite = "data/evenbrite.xlsx"
eag = "data/eag.xlsx"
empresarios_tv = "data/talento_visible_empresas.xlsx"
alta_gerencia = "data/alta_gerencia.xlsx"
lidera = "data/lidera.xlsx"
otros = "data/otros.xlsx"
influencers = "data/influencers.xlsx"
medios_comunicacion = "data/medios_comunicacion.xlsx"
letras_vanguardia_medellin = "data/letras_vanguardia_medellin.xlsx"
letras_vanguardia_bogota = "data/letras_vanguardia_bogota.xlsx"
veni_te_leo = "data/veni_te_leo.xlsx"
red_manos = "data/red_manos.xlsx"

In [6]:
# Bases de datos de comunicaciones
com_proveedores = ComunicationData("output/proveedores.csv")
com_aliados = ComunicationData("output/aliados.csv")
com_amvis = ComunicationData("output/amigos_visibles.csv")
com_donantes = ComunicationData("output/donantes.csv")
com_empresarios = ComunicationData("output/empresarios.csv")
com_tutores = ComunicationData("output/tutores_consultores.csv")
com_colaboradores = ComunicationData("output/colaboradores.csv")
com_beneficiarios = ComunicationData("output/beneficiarios.csv")
com_medios_comunicacion = ComunicationData("output/medios_comunicacion.csv")
com_influencers = ComunicationData("output/influencers.csv")

## Proveedores

In [5]:
proveedores = SheetData(amigos_visibles, "1.PROVEEDORES", "Organización", "Correo 1")
com_proveedores.append(proveedores.filter("Proveedores", ["proveedor"], "Tipo ", "ENCARGADO/A"))

## Aliados

In [6]:
aliados1 = SheetData(aliados, "HOJA2", "Nombre", "Correo")
com_aliados.append(aliados1.filter("Aliados", ["aliad", "soci"], "Tipo", "Entidad"))

In [7]:
aliados2 = SheetData(connect, "ORG ALIADAS", "NOMBRE", "CORREO ELECTRÓNICO")
com_aliados.append(aliados2.filter("Aliados", other="ORGANIZACIÓN O EMPRESA"))

In [8]:
aliados3 = SheetData(afrus, "18-09-2024-Personas-24-11-2023-", "Nombre completo", "Correo electrónico")
com_aliados.append(aliados3.filter("Aliados", ["aliad"], "Campo personalizado: otro"))

In [9]:
aliados4 = SheetData(evenbrite, "Sheet 1", "Nombre completo", "Correo electrónico de comprador")
com_aliados.append(aliados4.filter("Aliados", ["aliad"], "Tipo"))

In [10]:
aliados5 = SheetData(lidera, "Músicos", "Nombre", "Correo")
com_aliados.append(aliados5.filter("Aliados", other="Cargo"))

In [11]:
aliados6 = SheetData(lidera, "Actores Actrices", "Nombre", "Correo")
com_aliados.append(aliados6.filter("Aliados", other="Cargo"))

In [12]:
aliados7 = SheetData(lidera, "Cine ", "Nombre", "Correo")
com_aliados.append(aliados7.filter("Aliados", other="Cargo"))

In [13]:
aliados8 = SheetData(lidera, "Académicos", "Nombre", "Correo")
com_aliados.append(aliados8.filter("Aliados", other="Cargo"))

In [14]:
aliados9 = SheetData(lidera, "Artes Visuales", "Nombre", "Correo")
com_aliados.append(aliados9.filter("Aliados", other="Cargo"))

In [8]:
aliados10 = SheetData(otros, "Imaginarius", "Nombre", "Correo")
com_aliados.append(aliados10.filter("Aliados", other="Otro"))

## Amigos Visibles

In [15]:
donaciones = SheetData(donaciones_av, "Hoja 1", "Nombre completo", "Correo electrónico")
com_amvis.append(donaciones.filter("Amigos Visibles"))

## Donantes

In [16]:
donantes1 = SheetData(aliados, "HOJA2", "Nombre", "Correo")
com_donantes.append(donantes1.filter("Donantes", ["donant"], "Tipo", "Entidad"))

## Empresarios

In [17]:
empresarios1 = SheetData(eag, "Participantes ", "Nombre completo", "CORREO")
com_empresarios.append(empresarios1.filter("Empresarios", ["empresa", "consultor", "tecno", "banca"], "GRUPO"))

In [18]:
empresarios2 = SheetData(empresarios_tv, "Hoja1", "Nombre", "Correo")
com_empresarios.append(empresarios2.filter("Empresarios", other="Empresa"))

In [19]:
empresarios3 = SheetData(connect, "ORG CONNECT", "NOMBRE", "CORREO ELECTRÓNICO")
com_empresarios.append(empresarios3.filter("Empresarios", other="NOMBRE DE LA EMPRESA"))

In [20]:
empresarios4 = SheetData(evenbrite, "Sheet 1", "Nombre completo", "Correo electrónico de comprador")
com_empresarios.append(empresarios4.filter("Empresarios", ["empresa"], "Tipo"))

## Tutores y consultores

In [21]:
tutores1 = SheetData(amigos_visibles, "3. TUTORES", "Nombre", "Correo 1")
com_tutores.append(tutores1.filter("Tutores y consultores"))

In [22]:
tutores2 = SheetData(eag, "Base de datos de los tutores", "TUTOR", "CORREO ELECTRONICO")
com_tutores.append(tutores2.filter("Tutores y consultores"))

In [23]:
tutores3 = SheetData(evenbrite, "Sheet 1", "Nombre completo", "Correo electrónico de comprador")
com_tutores.append(tutores3.filter("Tutores y consultores", ["tutor", "consultor"], "Tipo"))

In [24]:
tutores4 = SheetData(amigos_visibles, "2. EQUIPO MV, JUNTA,ASAMBL ", "Nombre", "Correo 1")
com_tutores.append(tutores4.filter("Tutores y consultores", ["consultor", "tutor"], "Tipo"))

## Colaboradores

In [25]:
colaboradores1 = SheetData(amigos_visibles, "2. EQUIPO MV, JUNTA,ASAMBL ", "Nombre", "Correo 1")
com_colaboradores.append(colaboradores1.filter("Colaboradores", ["colaborador"], "Tipo"))

In [26]:
colaboradores2 = SheetData(evenbrite, "Sheet 1", "Nombre completo", "Correo electrónico de comprador")
com_colaboradores.append(colaboradores2.filter("Colaboradores", ["colaborador"], "Tipo"))

## Beneficiarios

In [27]:
beneficiarios1 = SheetData(amigos_visibles, "4. ORGANIZACIONES MINGALAB", "Nombre de la organización", "Correo 1")
com_beneficiarios.append(beneficiarios1.filter("Beneficiarios", other="Nombre del contacto"))

In [28]:
beneficiarios2 = SheetData(amigos_visibles, "4. PODER PACÍFICO", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios2.filter("Beneficiarios", other="Tipo"))

In [29]:
beneficiarios3 = SheetData(amigos_visibles, "5. POT ÉTNICA MDP, CE,MBA,MERC", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios3.filter("Beneficiarios", other="Tipo "))

In [30]:
beneficiarios4 = SheetData(amigos_visibles, "6. FJCP I Y II", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios4.filter("Beneficiarios", other="Tipo "))

In [31]:
beneficiarios5 = SheetData(amigos_visibles, "6. MIT", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios5.filter("Beneficiarios", other="Tipo "))

In [32]:
beneficiarios6 = SheetData(amigos_visibles, "6. ESCUELA SALUD PUB", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios6.filter("Beneficiarios"))

In [33]:
beneficiarios7 = SheetData(amigos_visibles, "8. FOCO", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios7.filter("Beneficiarios", other="Tipo "))

In [34]:
beneficiarios8 = SheetData(evenbrite, "Sheet 1", "Nombre completo", "Correo electrónico de comprador")
com_beneficiarios.append(beneficiarios8.filter("Beneficiarios", ["benefici"], "Tipo"))

In [35]:
beneficiarios9 = SheetData(alta_gerencia, "PARTICIPANTES ESC. ALTA GERENCI", "NOMBRES Y APELLIDOS DEL/LA PARTICIPANTE", "CORREO ELECTRÓNICO DEL/LA PARTICIPANTE")
com_beneficiarios.append(beneficiarios9.filter("Beneficiarios"))

c:\Users\Josue\Desktop\FVDN_DB\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [36]:
beneficiarios10 = SheetData(lidera, "Danza", "Nombre", "Correo")
com_beneficiarios.append(beneficiarios10.filter("Beneficiarios", other="Cargo"))

In [37]:
beneficiarios11 = SheetData(amigos_visibles, "7. DALE ", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios11.filter("Beneficiarios", other="Tipo "))

c:\Users\Josue\Desktop\FVDN_DB\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell E631 is marked as a date but the serial value 3117527666 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [38]:
beneficiarios12 = SheetData(amigos_visibles, "7. MUJERES LIDERES CARTAGENA", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios12.filter("Beneficiarios", other="Tipo "))

In [39]:
beneficiarios13 = SheetData(amigos_visibles, "8. POTENCIA ÉTNICA AUDIOVISUAL", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios13.filter("Beneficiarios", other="Tipo de potenciales Amigos"))

c:\Users\Josue\Desktop\FVDN_DB\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [40]:
beneficiarios14 = SheetData(amigos_visibles, "8. PMB", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios14.filter("Beneficiarios", other="Tipo "))

In [41]:
beneficiarios15 = SheetData(amigos_visibles, "8. FOCO", "Nombre", "Correo 1")
com_beneficiarios.append(beneficiarios15.filter("Beneficiarios", other="Tipo "))

In [7]:
beneficiarios16 = SheetData(letras_vanguardia_medellin, "BASE LETRAS DE VANGUARDIA", "Nombre", "CORREO ELECTRÓNICO DEL/LA PARTICIPANTE")
com_beneficiarios.append(beneficiarios16.filter("Beneficiarios", other="NOMBRE DEL TITULO OBTENIDO"))

In [8]:
beneficiarios17 = SheetData(veni_te_leo, "PARTICIPANTES ESCUELA VEC", "NOMBRES", "CORREO ELECTRÓNICO DEL/LA PARTICIPANTE")
com_beneficiarios.append(beneficiarios17.filter("Beneficiarios", other="NOMBRE DEL TITULO OBTENIDO"))

c:\Users\Josue\Documents\Trabajo\Manos Visibles\FVDN_DB\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [9]:
beneficiarios18 = SheetData(letras_vanguardia_bogota, "PARTICIPANTES ESCUELA VEC", "Nombre", "CORREO ELECTRÓNICO DEL/LA PARTICIPANTE")
com_beneficiarios.append(beneficiarios18.filter("Beneficiarios"))

c:\Users\Josue\Documents\Trabajo\Manos Visibles\FVDN_DB\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [10]:
beneficiarios19 = SheetData(red_manos, "BASE COMUNICACIONES PRELIMI (2)", "Nombre completo", "Correo Electrónico")
com_beneficiarios.append(beneficiarios19.filter("Beneficiarios", other="CONVOCATORIA"))

## Medios de comunicación

In [11]:
medios1 = SheetData(medios_comunicacion, "Directores y Periodistas ", "NOMBRE", "DIRECCION")
com_medios_comunicacion.append(medios1.filter("Medios de comunicación", other="MEDIO"))

In [12]:
medios2 = SheetData(influencers, "Periodistas", "Nombre", "Correo")
com_medios_comunicacion.append(medios2.filter("Medios de comunicación", other="Cargo"))

## Influencers

In [ ]:
influencers1 = SheetData(influencers, "")

# Unir archivos y eliminar duplicados

In [11]:
def merge_data(files: list, output_file: str, ignore_emails: list = None) -> None:
    if ignore_emails is None:
        ignore_emails = []

    all_data = pd.DataFrame(columns=["Nombre", "Correo", "Otro", "Origen", "Categoria"])

    # Leer y procesar cada archivo CSV
    for file in files:
        # Leer el archivo CSV
        df = pd.read_csv(file)

        # Asegurar que el correo sea en minúsculas y el nombre en mayúsculas
        df["Correo"] = df["Correo"].str.lower()
        df["Nombre"] = df["Nombre"].str.upper()

        # Filtrar solo los correos que son válidos (contienen @) y no están en la lista de correos a ignorar
        df = df[df["Correo"].str.contains("@", na=False) & ~df["Correo"].isin(ignore_emails)]

        # Concatenar con el DataFrame acumulado
        all_data = pd.concat([all_data, df], ignore_index=True)

    # Eliminar duplicados basados en la columna "Correo"
    unique_data = all_data.drop_duplicates(subset=["Correo"])

    # Guardar el resultado en un archivo CSV
    unique_data.to_csv(output_file, index=False)


In [12]:
input_files = [
    "donantes.csv",
    "amigos_visibles.csv",
    "empresarios.csv",
    "proveedores.csv",
    "aliados.csv",
    "tutores_consultores.csv",
    "colaboradores.csv",
    "beneficiarios.csv",
    "medios_comunicacion.csv"
]

ignore_emails = [
    "tecnologia@manosvisibles.org",
    "proyectos@manosvisibles.org",
    "convocatoria@manosvisibles.org",
    "convocatorias@manosvisibles.org",
    "legal@manosvisibles.org",
    "pmz@manosvisibles.org",
    "marlasso70@hotmail.com",
    "ricardoolmostorres@hotmail.com",
    "hectorhvo51@hotmail.com",
    "hochi_ming@hotmail.com",
    "Gracielaselaimen@gmail.com",
    "Felipe.viveros.m@gmail.com",
    "ml.arroyo1569@uniandes.edu.co",
    "ravila@eltiempo.com",

    "antiguo - @univalle.edu.co",
],

ignore_names = [
    "SALOMON DIAZ",
    "ALEJANDRA TABORDA MENA",
    "YEIMAR ANDRÉS RIVAS CABRERA",
    "NICOL JHOANA GARCES PLAZAS",
    "SARAY DANIELA VIVAS QUIÑONES",
    "JENNY VIVIANA ARBOLEDA GRUESO",
    "ÁNGELA MARÍA PARRA RIVERA",
    "MARIO  ANGULO  SANCLEMENTE",
    "MARIA CAMILA HURTADO MURILLO",
    "JEAN EDUARD CAICEDO ZAMORA",
    "YEINI VANESSA RIASCOS MONTOYA",
    "CLAUDIA YOHANA CUERO VALENCIA"
]

problematics = [
    "LAURA VICTORIA CAICEDO LASSO",
    "UNIVERSIDAD DE LOS ANDES",
    "VILLALON ENTRETENIMIENTO CREATIVO SAS",
    "ALEXIS PLAY",
    "KOMBILESA MI ANDRIS PADILLA / KEILA REGINA MIRANDA",
    "VINCE BALANTA",
    "MARTHA LUCÍA HERNÁNDEZ GARCÍA",
    "NATHALIA GIRALDO DIEZ",
    "ANGELICA MAYOLO ROGER MINA",
    "NEGRITA FILMS SAS",
    "PONTIFICA UNIVERSIDAD JAVERIANA",
    "RACHER JANE MINA",
    "JOSE ALEXANDER QUIÑONES CUERO",
    "LAURA VICTORIA CAICEDO LASSO",
    "VÍCTOR GUILLERMO CORTES RODRIGUEZ",
    "DIANA CAROLINA FUENTES RAMOS",
    "INDIANA PRADO TORRES",
    "KATHERINE MENA CASTRO",
    "LILIANA OCORO",
    "ALBERT MAURICIO MENA CHAVERRA",
    "MARIANGELES VENTE REBOLLEDO",
    "LIVIS ANDRES ALFONSO MINA",
    "YAIR ALFONZO CABESA HUILA",
    "SOLAY AMPARO GONZÁLEZ ANGULO",
    "MARIEN BONILLA PRECIADO",
    "EDUARDO ENRIQUE HALABY RENGIFO",
    "MARÍA ANGELA HURTADO VIAFARA",
    "PEDRO LUIS GUTIÉRREZ GONZÁLEZ",
    "RAMON CUESTA VALENCIA",
    "YAIRTON LLOREDA CASTRO",
    "PRESBÍTERO  HÉCTOR FERNEY VERGARA",
    "EDWAR MENA ROMAÑA",
    "HEMIRO BORJA PALACIOS",
    "DAFNY AMADEO PALACIOS MENA",
    "STELLA ROCIO RAMIREZ",
    "MILEIDY VANESA ARROYO CHAVERRA",
    "JONATHAN JOEL BANGUERA CORTES",
    "NELLY PEÑARANDA",
    "ANDRÉS LOPÉZ LÓPEZ",
    "FRANK SOLANO",
    "CIRO HERNÁNDEZ",
    "ANDRÉS MARÍN MARTÍNEZ",
    "ALFONSO OSPINA",
    "IGNACIO GREIFFENTEIN",
    "JOSÉ LUIS GARCÍA",
    "JORGE ESPINOSA",
    "JUAN CARLOS IRAGORRI",
    "MARIO JURSICH DURÁN",
    "JORGE LESMES MOGOLLÓN",
    "FREDY BOHORQUEZ",
    "RICHARD EMLIN",
    "INDALESIO CASTELLANOS",
    "YOLANDA RUÍZ",
    "MARÍA ISABEL RUEDA",
    "ALEX ALDANA",
    "RICARDO BERMÚDEZ",
    "QUACK DE COLOMBIA S.A.S",
    "UNIDAD INDÍGENA DEL PUEBLO AWÁ “UNIPA”",
    "JORGE ELIECER SERNA AGUDELO",
    "MANUEL FRANCISCO VIVEROS",
    "DIANA FERNANDA RIASCOS SALAZAR",
    "ANGIE ELISABETH CLEVEL MONTAÑO",
    "TAMARA LORETTA RENTERÍA CAICEDO",
    "ANDRÉS FABIÁN HURTADO GUERRERO",
    "ANDRÉS MAURICIO ACOSTA",
    "JHOANNA VALOYES /YADIRA RIVERA NAUFFAL",
    "LINA MARÍA CORREA MOLINA",
    "DANIER DARIO DIAZ PALACIOS",
    "MARTHA LUCÍA GARCÍA CUÉLLAR",
    "JUANY ANDREA RENTERIA RAMIREZ",
    "ANA JIMENA VALENCIA PRECIADO",
    "WILLIAN CUESTA IBARGUEN",
    "RODRIGO ACEVEDO",
    "JUAN CAMILO AMAR CABRERA",
    "FRANCISCO JAVIER FLOREZ BOLIVAR",
    "LAURA SOFÍA MICOLTA REYES",
    "MAVIL LISETH CABRERA MOSQUERA",
    "MIGUEL ÁNGEL GARCIA SÁNCHEZ",
    "KELLY HURTADO GAMBOA",
    "WIL MER HERIBERTO ORJUELA VALENCIA",
    "GLORIA ISABEL MONCAYO GONGORA",
    "TORRES RIOS YAILY KARINA",
    "LUIS CARLOS MARÏN GUANGA",
    "LUCY VALENTINA HERRERA RENTERÍA",
    "BRENDA YULIETH AMU ROJAS",
    "YILMAR MARTÍNEZ"
]

input_files = [f"output/{f}" for f in input_files]
merge_data(input_files, "datos_combinados.csv", ignore_emails=ignore_emails)

# Normalizar nombres

In [1]:
import csv
import unicodedata
import re

def normalize_column(input_csv, output_csv, column_name):
    def normalize_text(text):
        # Eliminar acentos y caracteres especiales
        text = unicodedata.normalize('NFD', text)
        text = text.encode('ascii', 'ignore').decode('utf-8')
        # Eliminar caracteres que no sean letras o espacios
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Eliminar espacios de más
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    with open(input_csv, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:

        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            row[column_name] = normalize_text(row[column_name])
            writer.writerow(row)

normalize_column("beneficiarios.csv", "Base Beneficiarios.csv", "Nombre")